In [31]:
# Importar bibioteca
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [47]:
BASE_URL = 'https://www.gruposhopmix.com/'

In [13]:
## obter lista de urls a partir do index
data  = pd.read_excel('./data/dados.xlsx' , sheet_name='global')

In [23]:
content = requests.get(data['tag'][0]).text
content

'<?xml version="1.0" encoding="utf-8"?>\n<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n  <url>\n    <loc>https://www.gruposhopmix.com/</loc>\n    <changefreq>always</changefreq>\n    <priority>1.0</priority>\n  </url>\n\n  \n  <url>\n    <loc>https://www.gruposhopmix.com/pagina/sobre-o-gruposhopmix.html</loc>\n    <changefreq>always</changefreq>\n    <priority>0.7</priority>\n  </url>\n  \n  <url>\n    <loc>https://www.gruposhopmix.com/pagina/endereco-e-funcionamento.html</loc>\n    <changefreq>always</changefreq>\n    <priority>0.7</priority>\n  </url>\n  \n  <url>\n    <loc>https://www.gruposhopmix.com/pagina/dropshipping.html</loc>\n    <changefreq>always</changefreq>\n    <priority>0.7</priority>\n  </url>\n  \n  <url>\n    <loc>https://www.gruposhopmix.com/pagina/planilha-de-produtos.html</loc>\n    <changefreq>always</changefreq>\n    <priority>0.7</priority>\n  </url>\n  \n  <url>\n    <loc>https://www.gruposhopmix.com/pagina/como-baixar-imagens-do-catalogo.html<

In [37]:
### extracts all loc tags from content
soup = bs(content, 'html.parser')
all_locs = soup.find_all('loc')
urls = pd.DataFrame(all_locs)

c:\Users\bcesa\anaconda3\envs\shopmix_scraper\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [53]:
### remove keywords from urls
keywords = ['pagina','sao-paulo', 'todos-os-produtos', 'os-mais-vendidos', 'novidades']
urls = urls[~urls[0].str.contains('|'.join(keywords))]
## drop lines that is equal to the base url
urls = urls.loc[urls[0] != BASE_URL]

urls.reset_index(inplace=True, drop=True)

In [131]:
urls[0][540]

'https://www.gruposhopmix.com/produto/pulseira-guia-infantil-cordao-seguranca-da-crianca-verde'

In [125]:
prod_content = requests.get(urls[0][540]).text

In [126]:
prod_content_soup = bs(prod_content, 'html.parser')

In [127]:
tags = ["title", 'nome-produto titulo cor-secundaria']

prod_content_soup.find_all(tags)

[<title>Pulseira Guia Infantil Cordão Segurança da Criança Verde - Catálogo GrupoShopMix</title>]

In [132]:
## salvar prod_content_soup em um arquivo texto

with open('./data/produtos.txt', 'w') as f:
    f.write(str(prod_content_soup.contents[3]))

In [62]:
prod_content_soup

### extrair dados de cada produto nome, SKU, preço, descrição, categoria, imagem, url



[]

In [76]:
str(prod_content_soup.contents[3])

'<html lang="pt-br">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">\n<title>Caneca Térmica Com Tampa 380 ml Mixer A Pilha - Catálogo GrupoShopMix</title>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta content="Loja Integrada" name="generator">\n<link href="https://cdn.awsli.com.br/" rel="dns-prefetch"/>\n<link href="https://cdn.awsli.com.br/" rel="preconnect"/>\n<link href="https://fonts.googleapis.com" rel="preconnect"/>\n<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>\n<meta content="https://www.gruposhopmix.com/caneca-termica-com-tampa-380-ml-mixer-a-pilha" property="og:url"/>\n<meta content="website" property="og:type"/>\n<meta content="GrupoShopMix" property="og:site_name"/>\n<meta content="pt_BR" property="og:locale"/>\n<!-- Metadata para o facebook -->\n<meta content="website" property="og:type"/>\n<meta content="Caneca Térmica Com Tampa 380 ml Mixer A Pilha" pro

In [140]:

def extract_data_from_html(soup):
    
    
    nome = soup.find('meta', property='og:title')['content']
    codigo = soup.find('meta', attrs={'name': 'twitter:data1'})['content']
    preco = soup.find('strong', attrs = 'preco-promocional cor-principal titulo')['data-sell-price']
    disponibilidade = soup.find('meta', attrs={'name': 'twitter:data2'})['content']
    descricao = soup.find('meta', attrs={'name': 'description'})['content']
    disponibilidade_entrega = soup.find('span', class_='disponibilidade disp-entrega').find('b').text
    estoque = soup.find('b', class_='qtde_estoque').text
    imagens = [img['src'] for img in soup.find_all('img') if 'produto' in img['src']]
    
    return {
        'nome': nome,
        'código': codigo,
        'preço': preco,
        'descrição': descricao,
        'disponibilidade': disponibilidade,
        'disponibilidade_entrega': disponibilidade_entrega,
        'estoque': estoque,
        'imagens': imagens
    }

In [139]:
prod_content_soup.find('b', class_='qtde_estoque').text

'120'

In [141]:
extract_data_from_html(prod_content_soup)

{'nome': 'Pulseira Guia Infantil Cordão Segurança da Criança Verde',
 'código': 'STO322VD-167',
 'preço': '26.90',
 'descrição': 'Pulseira Guia Infantil Cordão Passeio Segurança da CriançaDescriçãoPode ser utilizado em qualquer lugar, em passeios, viagens, shoppings, etc.Evita que a criança atravesse acidentalmente ou corra para locais não seguros como ruas e avenidas.Proporciona muito mais segurança pra seus passeios com sua criança.Especificações/Características do Produto:- Material: Fio de aço inoxidável Algodão e Plástico expansível;- Cor: Verde- Idade aplicável: 1 aos 10 anos;- Comprimento Máximo: 2mItem Incluso:- 1 Pulseira Guia Infantil',
 'disponibilidade': 'Disponível',
 'disponibilidade_entrega': 'Imediata',
 'estoque': '120',
 'imagens': ['https://cdn.awsli.com.br/64x50/1080/1080773/produto/2240815694b3697dd42.jpg',
  'https://cdn.awsli.com.br/64x50/1080/1080773/produto/224081569502bf3354e.jpg',
  'https://cdn.awsli.com.br/64x50/1080/1080773/produto/224081569edfce15996.jpg'